In [1]:
import pandas as pd
import numpy as np
import os
import itertools
from itertools import tee, combinations, permutations
from progressbar import ProgressBar
import datetime
from sklearn.metrics import mean_squared_error, accuracy_score
import time
import calendar
from statistics import median

import math

os.chdir('../')

In [20]:
os.chdir('./Sprint4')

In [21]:
data_train = pd.read_csv('./data/2018-train.csv', encoding = "ISO-8859-1",
                         error_bad_lines=False, dtype={'event org:resource':str})

In [24]:
data_train['event time:timestamp'] = pd.to_datetime(data_train['event time:timestamp'])

In [26]:
data_train['event time:timestamp'].sort_values

<bound method Series.sort_values of 0         2014-04-05 00:00:00.000
1         2014-04-05 00:00:00.000
2         2014-04-05 00:00:00.000
3         2014-04-05 00:00:00.000
4         2014-04-05 00:00:00.000
                    ...          
2002740   2018-01-19 12:27:24.824
2002741   2018-01-19 12:51:34.000
2002742   2018-01-19 12:57:05.000
2002743   2018-01-19 13:02:32.000
2002744   2018-01-19 13:03:02.000
Name: event time:timestamp, Length: 2002745, dtype: datetime64[ns]>

In [28]:
data_test = pd.read_csv('./data/2017-test.csv', encoding = "ISO-8859-1", 
                        error_bad_lines=False, dtype={'event org:resoure':str})

data_test['event time:timestamp'] = pd.to_datetime(data_test['event time:timestamp'])

df2_train = data_train
df2_test = data_test

# Concatenate the datasets
df2 = pd.concat([df2_train, df2_test])

# Sort the values and reset the index accordingly
df2 = df2.sort_values(by=['event time:timestamp'])
df2 = df2.reset_index(drop=True)

# Create dataframe with first events of each case in chronological order
df_cases_first_event = df2.drop_duplicates(subset='case concept:name', keep='first')
df_cases_first_event = df_cases_first_event.reset_index(drop=True)

# Split the cases in 80-20
index80 = int(len(df_cases_first_event) * 80 / 100 - 1)
df_cases_train = df_cases_first_event.loc[:index80]
df_cases_test = df_cases_first_event.loc[index80 + 1:]

# Training and test set complete
df_train_new = df2.loc[df2['case concept:name'].isin(list(df_cases_train['case concept:name']))]
df_test_new = df2.loc[df2['case concept:name'].isin(list(df_cases_test['case concept:name']))]

# Setting the 'now' (as the last event of the last case in the training set)
# now = df_train_new[df_train_new['case concept:name']==df_cases_train.iloc[-1]\
#    ['case concept:name']].iloc[-1]['event time:timestamp']
if 'case LoanGoal' in df2_train:
    now = pd.Timestamp('2016-12-08')
else:
    now = pd.Timestamp('2017-12-12')

# Training and test set before 'now'
df_train_new_now = df_train_new.loc[(df_train_new['event time:timestamp'] <= now)]
df_test_new_now = df_test_new.loc[(df_test_new['event time:timestamp'] <= now)]

data_train = df_train_new_now
data_test = df_test_new_now

In [57]:
len(data_test) / (len(data_train) + len(data_test))

0.1790913589663266

In [ ]:
(eventID, caseID, event_name, transition, stamp) = ("eventID ", "case concept:name",
                                                    "event concept:name", "event lifecycle:transition",
                                                    "event time:timestamp")
## ADD 
if 'case LoanGoal' in data_train.columns:
    data_train['case LoanGoal'] = data_train['case LoanGoal'].apply(
        lambda x: 'Other - see explanation' if x == 'Other, see explanation' else x)
    
    data_test['case LoanGoal'] = data_test['case LoanGoal'].apply(
        lambda x: 'Other - see explanation' if x == 'Other, see explanation' else x)
    datetimeFormat = '%Y-%m-%d %H:%M:%S.%f'

    pt1, pt2, pt3 = 4, 8, 12
else:  # check if raod data
    datetimeFormat = '%Y-%m-%d'

    pt1, pt2, pt3 = 2, 3, 5

datetimeFormat = '%Y-%m-%d %H:%M:%S.%f'
    
# Sorting the frame chronologically and per case.
data_train = data_train.sort_values(by=[caseID, stamp])
data_test = data_test.sort_values(by=[caseID, stamp])

# New .csv file with the fixed sorting and timestamp
data_train.to_csv("./fixed_train.csv")
data_test.to_csv("./fixed_test.csv")
# Creating a dictionary file where the case names (IDs) are the keys and the values are a list of two sublists.
# The first sublist contains all event names of a case, sorted chronologically.
# The second sublist contains all event time stamps of a case, sorted chronologically.

pt1 = data_train.columns.get_loc('case concept:name')
pt2 = data_train.columns.get_loc('event concept:name')
pt3 = data_train.columns.get_loc('event time:timestamp')
#

In [45]:
# file = open('fixed.csv', 'r')
log = dict()
# print('making dictionary from fixed file')
with open('./fixed_train.csv', 'r') as f:
    next(f)

    for line in f:
        line = line.strip()
        if len(line) == 0:
            continue
        parts = line.split(',')
        caseid = parts[pt1+1]
        task = parts[pt2+1]
        timestamp = parts[pt3+1]

        if caseid not in log:
            log[caseid] = [[], []]

        log[caseid][0].append(task)
        log[caseid][1].append(timestamp)

f.close()

log_test = dict()
# print('making test dictionary from fixed file')
with open('./fixed_test.csv', 'r') as f:
    next(f)

    for line in f:
        line = line.strip()
        if len(line) == 0:
            continue
        parts = line.split(',')
        caseid = parts[pt1+1]
        task = parts[pt2+1]
        timestamp = parts[pt3+1]

        if caseid not in log_test:
            log_test[caseid] = [[], []]

        log_test[caseid][0].append(task)
        log_test[caseid][1].append(timestamp)

f.close()

In [46]:
#data_train.loc[0][pt1]
log
#print(pt1)

{'0002505cb62792e4': [['calculate',
   'begin editing',
   'calculate',
   'finish editing',
   'begin editing',
   'calculate',
   'finish editing',
   'mail valid',
   'mail valid',
   'mail valid',
   'decide',
   'calculate',
   'finish editing',
   'mail income',
   'decide',
   'initialize',
   'finish editing',
   'performed',
   'initialize',
   'begin editing',
   'calculate',
   'calculate',
   'finish editing',
   'revoke decision',
   'performed',
   'decide',
   'initialize',
   'begin editing',
   'calculate',
   'calculate',
   'finish editing',
   'begin editing',
   'calculate',
   'finish editing',
   'calculate',
   'initialize',
   'begin editing',
   'initialize',
   'finish editing',
   'begin editing',
   'begin payment',
   'abort payment',
   'begin payment',
   'abort payment',
   'begin payment',
   'finish payment',
   'insert document',
   'initialize',
   'begin editing',
   'calculate',
   'remove document',
   'insert document',
   'initialize',
   'begi

In [47]:
data_train['event concept:name']

6763             calculate
17057        begin editing
17063            calculate
17065       finish editing
22136        begin editing
                ...       
1864466      begin editing
1864467          calculate
1864468    remove document
1864469    insert document
1864470               save
Name: event concept:name, Length: 1808158, dtype: object

In [48]:
# print('done making dictionaries')
# print('deleting fixed files')
#os.remove('./fixed_train.csv')
#os.remove('./fixed_test.csv')

# print('done')

### Event prediction

event_list = []  # list of lists that contains all event names for every case, where each sublist is a new case.
for case in log.keys():
    event_list.append((log[case][0]))

events_longest_event = max(map(len, event_list))  # longest event

pos_events = []  # list of lists which has all event names for every position, where each sublist is a new position
for i in range(events_longest_event):  # creates a list every iteration which is appended to the above list
    pos_events.append([])
    for case in event_list:
        if (len(case) - 1) >= i:
            pos_events[i].append(case[i])
        else:
            pass

cases = list(data_train[event_name].unique())  # list of all unique event names
event_frequency = []
index = 0
str_best = cases[0]
for i in range(len(pos_events)):  # counts how many times each event name occurs in every position
    for x in cases:
        best = pos_events[i].count(cases[0])
        current = pos_events[i].count(x)
        if current > best:
            best = current
            str_best = x
    event_frequency.append(str_best)  # only the variable with highest count is appended to the frequency list
    index += 1
#print('Event prediction done')
### Timestamp prediction

# list of lists that contains all timestamps for all events of a case, where every sublist is a new case
time_list = []
for case in log.keys():
    time_list.append((log[case][1]))

times_longest_event = max(map(len, time_list))  # longest event
# print("Longest event in train: ", times_longest_event)

time_list = []
for case in log_test.keys():
    time_list.append((log_test[case][1]))

times_longest_event_test = max(map(len, time_list))  # longest event

In [49]:
# print("Longest event in test: ", times_longest_event_test)

def timeDiff(tup, datetimeFormat):
    """The function calculates the difference between two timestamps and returns the absolute value."""

    # COMMENT OUT datetimeFormat = '%Y-%m-%d'
    diff = datetime.datetime.strptime(tup[0], datetimeFormat) \
           - datetime.datetime.strptime(tup[1], datetimeFormat)

    return abs(diff.days)

pbar = ProgressBar()

# list of lists that contains all event timestamps for every position, where each sublist is a new position
pos_times = []
print('[1/13]')
for i in pbar(range(times_longest_event)):  # new list is created each iteration
    pos_times.append([])
    for case in time_list:  # for every case a tuple created which contains the current and the next event time
        if (len(case) - 1) > i:
            event_tpl = (case[i], case[i + 1])
            #print(event_tpl)
            pos_times[i].append(timeDiff(event_tpl, datetimeFormat))  # the difference is calculated and saved in a list
        else:
            pass

[1/13]


100% |########################################################################|


In [50]:
time_frequency = []  # list that contains all predictions for every position
for i in pos_times:
    avg_pos = np.mean(i)  # the mean for every position is calculated and appended to the corresponding index
    if not math.isnan(avg_pos):  # a small check to avoid bugs because the last value is a NaN
        avg_pos = int(np.mean(i))
        time_frequency.append(avg_pos)
    else:
        time_frequency.append(avg_pos)
#print('Time prediction Done')

### Adding predictions to the file
# * To the test file
for case in log_test.keys():
    n_events = len(log_test[case][0])
    n_times = len(log_test[case][1])

    if len(log_test[case][0]) > times_longest_event:
        n_unknown = len(log_test[case][0]) - times_longest_event
        evs = event_frequency[:n_events]
        evs.extend(['Unknown' for one in range(0, n_unknown)])

        tms = time_frequency[:n_times]
        tms.extend([31 for one in range(0, n_unknown)])

        log_test[case].extend((evs, tms))
    else:
        log_test[case].extend((event_frequency[:n_events], time_frequency[:n_times]))

for case in log_test.keys():
    real_diff = []  # empty list for real time differences for each case
    for event, i in zip(log_test[case][1],
                        range(len(log_test[case][1]))):  # add for each event the difference between this
        if (len(log_test[case][1]) - 1) > i:
            this = event
            following = log_test[case][1][i + 1]
            real_time_tup = (this, following)
            real_diff.append(timeDiff(real_time_tup, datetimeFormat))
        else:
            real_diff.append(0)
    log_test[case].extend([real_diff])

case_names = []
event_names = []
timestamp = []
p_event = []
p_timestamp = []
time_diff = []

for i in log_test.keys():
    for x in range(len(log_test[i][0])):
        case_names.append(i)
        # it = iter(log_test[i])
        # the_len = len(next(it))
        # if not all(len(l) == the_len for l in it):
        #     print(log_test[i])
        #     raise ValueError('not all lists have same length!')
        event_names.append(log_test[i][0][x])
        timestamp.append(log_test[i][1][x])
        p_event.append(log_test[i][2][x])
        p_timestamp.append(log_test[i][3][x])
        time_diff.append(log_test[i][4][x])

frame_dict = {'Case_ID': case_names, 'Event_Name': event_names,
              'TimeStamp': timestamp, 'Real_TimeDiff': time_diff, 'Baseline_Event': p_event,
              'Baseline_TimeDiff': p_timestamp}

predicted_df = pd.DataFrame.from_dict(frame_dict)

In [51]:
event_real = np.array(predicted_df['Event_Name'])[1:]

time_real = np.array(predicted_df['Real_TimeDiff'])

event_pred_baseline = np.array(predicted_df['Baseline_Event'])[:-1]
# event_pred_baseline = event_pred_baseline[:-1]
time_pred_baseline = np.array(predicted_df['Baseline_TimeDiff'])

acc_baseline = accuracy_score(event_real, event_pred_baseline)
print('Accuracy for event prediction BASELINE: {}%'.format(round(acc_baseline, 2) * 100))

Accuracy for event prediction BASELINE: 15.0%


In [52]:
if pd.Series(time_pred_baseline).isnull().values.any():
    time_pred_baseline = [i if (not np.isnan(i)) else 5. for i in time_pred_baseline]
                          #[i for i in time_pred_baseline if (not i.isnull()) else 5]
    
pd.Series(time_pred_baseline).isnull().values.any()

False

In [53]:
rms_baseline = np.sqrt(mean_squared_error(time_real, time_pred_baseline))
print('Root mean squared error for time difference prediction BASELINE: {}'.format(round(rms_baseline, 2)))

Root mean squared error for time difference prediction BASELINE: 13.03


In [54]:
len(time_real), len(time_pred_baseline)

(394472, 394472)

True